In [ ]:
# loading the API keys in environment variables
import os, json
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.tools import tool

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
# If you have langsmith access, run the following to trace and debuy your agent

from langsmith import Client
client = Client()

os.environ['LANGCHAIN_PROJECT'] = "contoso-evaluate-agent"

In [ ]:
openai.api_key = os.environ['OPENAI_API_KEY']
openai.api_base = os.environ['OPENAI_API_BASE']
openai.api_type = os.environ['OPENAI_API_TYPE']
openai.api_version = os.environ['OPENAI_API_VERSION']

chat_model = os.environ['GPT4_MODEL_NAME']
embedding_model = os.environ['EMBEDDING_MODEL_NAME']
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(model=embedding_model, chunk_size=1)
vector_store_endpoint: str = os.environ['AZURE_COGNITIVE_SEARCH_ENDPOINT']
vector_store_password: str = os.environ['AZURE_COGNITIVE_SEARCH_KEY']
index_name: str = "contoso-coffee-index"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_endpoint,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [ ]:
import time
# from langchain.schema.agent import AgentFinish
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor


chain = prompt | model | OpenAIFunctionsAgentOutputParser()

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

# Initialize the agent with return_intermediate_steps=True to return agent's trajectory for evaluation
agent_executor = AgentExecutor(agent=agent_chain, tools=tool_list, verbose=False, memory=memory, return_intermediate_steps=False)

from langchain.evaluation import load_evaluator
evaluator = load_evaluator("trajectory")

result = agent_executor.invoke({"input": "what are your famous french coffee and bakery items?"})

evaluation_result = evaluator.evaluate_agent_trajectory(
        prediction=result["output"],
        input=result["input"],
        agent_trajectory=result["output"],
    )

